In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

In [2]:
from platform import python_version

print(python_version())

3.5.2


In [3]:
# create the session
conf = SparkConf().setMaster("local").setAppName("My app")
# create the context
#sc = pyspark.SparkContext(conf=conf)
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

# Extract words having length greater than 4

In [9]:
def func(lines):
    
    p = []
    
    
    for l in lines:
        
        if len(l)>4:
    
            p.append(l.lower())
        
    
    return p


# Print Input File Content

In [50]:
inputfile = sc.textFile("./file1.txt")
inputfile.collect()

['Horrid Henry’s hound hunts in the massive Murree mountains. While silly stupid Samuel’s dark dreadful dragon likes to hunt in',
 'skies.',
 'Horrid Henry’s hound and Samuel’s dreadful dragon Dany are fast friends and like to hunt and play together. They call themselves',
 'fantastic fanciful foursome.']

# Generate Key/Value pairs such that trigrams of length greater than 4 being the value while the starting alphabet of the first word in the trigram being the key

In [13]:
inputfile = sc.textFile("./file1.txt")
inputfile.flatMap(lambda line: line.split(".")).map(lambda line: line.split(" ")).map(func).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:]))).map(lambda x:(x[0][0], x)).reduceByKey(lambda x, y: (x, y)).collect()

[('s',
  (((('silly', 'stupid', 'samuel’s'), ('stupid', 'samuel’s', 'dreadful')),
    ('samuel’s', 'dreadful', 'dragon')),
   ('samuel’s', 'dreadful', 'dragon'))),
 ('h',
  ((((((('horrid', 'henry’s', 'hound'), ('henry’s', 'hound', 'hunts')),
       ('hound', 'hunts', 'massive')),
      ('hunts', 'massive', 'murree')),
     ('horrid', 'henry’s', 'hound')),
    ('henry’s', 'hound', 'samuel’s')),
   ('hound', 'samuel’s', 'dreadful'))),
 ('d',
  ((('dreadful', 'dragon', 'likes'), ('dreadful', 'dragon', 'friends')),
   ('dragon', 'friends', 'together'))),
 ('m', ('massive', 'murree', 'mountains')),
 ('w', ('while', 'silly', 'stupid')),
 ('f', ('fantastic', 'fanciful', 'foursome'))]

# Sum up the number of values corresponding each key

In [21]:
inputfile = sc.textFile("./file1.txt")
inputfile.flatMap(lambda line: line.split(".")).map(lambda line: line.split(" ")).map(func).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:]))).map(lambda x:(x[0][0], 1)).reduceByKey(lambda accum, n: accum + n).collect()

[('s', 4), ('h', 7), ('d', 3), ('m', 1), ('w', 1), ('f', 1)]

# Generate Trigrams such that all words in a trigram start with the same word

In [39]:
inputfile = sc.textFile("./file1.txt")
inputfile.flatMap(lambda line: line.split(".")).map(lambda line: line.split(" ")).map(func).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:]) if (x[0][0] == x[1][0] == x[2][0]))).collect()

[('horrid', 'henry’s', 'hound'),
 ('henry’s', 'hound', 'hunts'),
 ('massive', 'murree', 'mountains'),
 ('silly', 'stupid', 'samuel’s'),
 ('horrid', 'henry’s', 'hound'),
 ('fantastic', 'fanciful', 'foursome')]

# Generate Key/Value pairs such that trigram being the key while the number of times it appears is the value

In [48]:
inputfile = sc.textFile("./file1.txt")
inputfile.flatMap(lambda line: line.split(".")).map(lambda line: line.split(" ")).map(func).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:]) if (x[0][0] == x[1][0] == x[2][0]))).map(lambda x:(x, 1)).reduceByKey(lambda accum, n: accum + n).collect()

[(('silly', 'stupid', 'samuel’s'), 1),
 (('fantastic', 'fanciful', 'foursome'), 1),
 (('massive', 'murree', 'mountains'), 1),
 (('horrid', 'henry’s', 'hound'), 2),
 (('henry’s', 'hound', 'hunts'), 1)]

# Generate Key/Value pairs such that starting word of the trigram being the key while number of times it appears being the value

In [35]:
inputfile = sc.textFile("./file1.txt")
inputfile.flatMap(lambda line: line.split(".")).map(lambda line: line.split(" ")).map(func).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:]) if (x[0][0] == x[1][0] == x[2][0]))).map(lambda x:(x[0][0], 1)).reduceByKey(lambda accum, n: accum + n).collect()

[('s', 1), ('h', 3), ('m', 1), ('f', 1)]

# Count and print the number of three long consecutive words in a sentence that starts with the same english alphabet. We say that a word is long if it is greater than four alphabets.

In [36]:
inputfile = sc.textFile("./file1.txt")
output=inputfile.flatMap(lambda line: line.split(".")).map(lambda line: line.split(" ")).map(func).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:]) if (x[0][0] == x[1][0] == x[2][0]))).map(lambda x:(x[0][0], 1)).reduceByKey(lambda accum, n: accum + n).collect()

In [38]:
type(output), print (output)

[('s', 1), ('h', 3), ('m', 1), ('f', 1)]


(list, None)

In [40]:
count = 0
for o in output:
    count += o[1]

In [43]:
print ("Total count : ",count)

Total count :  6


# Print the number of three long consecutive words starting with each alphabet.

In [47]:
count = 0
for o in output:
    print (o[0].upper(),"=>",o[1])

S => 1
H => 3
M => 1
F => 1


# Print the number of times each combination of three long consecutive words occur in the file.

In [49]:
inputfile = sc.textFile("./file1.txt")
inputfile.flatMap(lambda line: line.split(".")).map(lambda line: line.split(" ")).map(func).flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:]) if (x[0][0] == x[1][0] == x[2][0]))).map(lambda x:(x, 1)).reduceByKey(lambda accum, n: accum + n).collect()

[(('silly', 'stupid', 'samuel’s'), 1),
 (('fantastic', 'fanciful', 'foursome'), 1),
 (('massive', 'murree', 'mountains'), 1),
 (('horrid', 'henry’s', 'hound'), 2),
 (('henry’s', 'hound', 'hunts'), 1)]